In [ ]:
import face_recognition
import cv2
import os
import csv
import numpy as np
import pafy
from PIL import Image
import glob
import json
from google import google
import string
import nltk

In [38]:
# Open the input movie file

url = 'https://www.youtube.com/watch?v=6ZfuNTqbHE8&t=64s'
vPafy = pafy.new(url)
play = vPafy.getbest()
video = play.download()

input_movie = cv2.VideoCapture(video)
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

In [39]:
nltk.download('punkt')

video_title = vPafy.title
video_title = video_title.lower()
video_title = video_title.translate(str.maketrans('', '', string.punctuation))

list = ["official", "trailer", "teaser","italiano", "movie", "hd"]

stop_words = set(list)

word_tokens = nltk.word_tokenize(video_title) 

print("tokenizzazione:")
print(word_tokens)
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 

print("stopping")
print(filtered_sentence)

stopped_string = ''
for w in filtered_sentence:
    stopped_string = stopped_string + w + " "

print("stringa per la ricerca:")
print(stopped_string)

search_results = google.search(stopped_string + "imdb")
link = search_results[0].link

print("Risultato della ricerca:")
print(link)

id = link[link.find("title/")+6:len(link)-1]
print("Id del film:")
print(id)

[nltk_data] Downloading package punkt to /home/daniele/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizzazione:
['marvel', 'studios', 'avengers', 'infinity', 'war', 'official', 'trailer']
stopping
['marvel', 'studios', 'avengers', 'infinity', 'war']
stringa per la ricerca:
marvel studios avengers infinity war 
Risultato della ricerca:
https://www.imdb.com/title/tt4154756/
Id del film:
tt4154756


In [40]:
import pandas as pd
known_faces = pd.read_csv('file.csv', sep=',',header=None).values

In [41]:
map = pd.read_csv('dict_file.csv', sep=',',header=None).values

In [42]:
face_locations = []
face_encodings = []
frame_number = 0

current_path = os.getcwd()
counter = 0
face_names = []

In [43]:
print("Inizio processamento")
while True:
    
    # Grab a single frame of video
    input_movie.set(1, frame_number)
    ret, frame = input_movie.read()
    frame_number += 20

    # Quit when the input video file ends
    if not ret:
        print("Fine")
        break

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)


    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)
        # name = None
        if True in match:
            index = match.index(True)
            name = map[index]
            face_names.append(name[1])
        # If you had more than 2 faces, you could make this logic a lot prettier
        # but I kept it simple for the demo

Inizio processamento
Fine


In [44]:
# All done!
input_movie.release()
cv2.destroyAllWindows()
# create_csv.CreateCsv(current_path + "/face_database/")
print (face_names)

['katherine_waterson', 'mark_ruffalo', 'mark_ruffalo', 'scarlett_johansson', 'scarlett_johansson', 'chris_pratt', 'tom_holland', 'robert_downeyjr', 'scarlett_johansson', 'benedict_cumberbatch', 'benedict_cumberbatch']


In [53]:
face_names = set(face_names)

TypeError: 'list' object is not callable

In [50]:
final_list = []
for face in face_names:
    name = face[0:face.find("_")]
    surname = face[face.find("_")+1:len(face)]
    final_list.append(name.capitalize() + " " + surname.capitalize())

In [51]:
json.dumps(final_list)

'["Robert Downeyjr", "Tom Holland", "Scarlett Johansson", "Chris Pratt", "Katherine Waterson", "Benedict Cumberbatch", "Mark Ruffalo"]'

# Interrogare PredictionIO

In [ ]:
#{ "items": ["tt0133240", "tt0153340"], 
#   "num": 4,
#   "categories" : ["Meg Ryan"]}

L'engine di predictionIO ascolta sulla porta 8000. Per conoscere l'ip dell'engine nella rete bridge create, digitare: <br>
    * $> docker inspect --format '{{ .NetworkSettings.IPAddress }}' <name or id container>

In [ ]:
import predictionio
#modificare l'ip del server di PredictionIO (guarda il readme in creazione rete)
engine_client = predictionio.EngineClient(url="http://<IP_Engine>:8000")
print(engine_client.send_query({"items": ["tt0133240"], # film simili a questo. E' l'id di imdb
                                "num": 4, # quanti film voglio che mi raccomandano
                                "categories" : ["Meg Ryan"]})) # filtro sugli attori